## Baseline Model: KNN on face keypoints dataset

In [1]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots

sns.set(style="darkgrid")  # default style

from PIL import Image
from classifier.data_loader import load_data
from classifier.k_nearest_neighbor import KNearestNeighbor


In [2]:
# Load the data

X_train, X_val, y_train, y_val, X_test = load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")


Image array shape: (7049, 96, 96), Label array shape: (7049, 30)
Image array shape: (1783, 96, 96)
X_train shape: (5639, 96, 96)
y_train shape: (5639, 30)
X_val shape: (1410, 96, 96)
y_val shape: (1410, 30)
X_test shape: (1783, 96, 96)


In [3]:
# reshape the data to 2D

X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

print(X_train.shape, X_val.shape, X_test.shape)

(5639, 9216) (1410, 9216) (1783, 9216)


In [4]:
# trian a knn classifier with only nose location
knn_nose = KNearestNeighbor()
#nose is the 20 and 21 index
y_train_nose = y_train[:, 20:22]
y_val_nose = y_val[:, 20:22]

print(y_train_nose.shape, y_val_nose.shape)


(5639, 2) (1410, 2)


In [5]:

knn_nose.train(X_train, y_train_nose)
losses = []

for k in [1, 3, 5, 10, 20, 50]:
    y_pred_nose = knn_nose.predict(X_val, k=k)
    squared_error = np.sum(np.power(y_pred_nose - y_val_nose, 2), axis=1)
    loss = np.sqrt(np.mean(squared_error))
    losses.append(loss)
    print(f"k: {k}, loss: {loss:.4f}")

/home/rcao/repos_linux/ucb_mids_w207_project/classifier/k_nearest_neighbor.py:69: RuntimeWarning: invalid value encountered in sqrt
  dists = np.sqrt(squared_sum)


k: 1, loss: 6.7430
k: 3, loss: 6.0847
k: 5, loss: 5.9584
k: 10, loss: 5.9438
k: 20, loss: 6.0399
k: 50, loss: 6.3284


The interpretation is on average, the model is off by 6 pixels in the x direction and 6 pixels in the y direction. Considering the range of the data is 0 to 96, this is a pretty good result.

In [16]:
# predicting all the points

# TODO: right now we have to drop all the points that have missing values, but we should not need to do so?
def get_clean_data(X, y):
    clean = ~np.isnan(y).any(axis=1)
    return X[clean, :], y[clean, :]

X_train_clean, y_train_clean = get_clean_data(X_train, y_train)
X_val_clean, y_val_clean = get_clean_data(X_val, y_val)

# trian a knn classifier with all the points
knn = KNearestNeighbor()
knn.train(X_train_clean, y_train_clean)
losses = []

for k in [1, 3, 5, 10, 20, 50]:
    y_pred = knn.predict(X_val_clean, k=k)
    squared_error = np.sum(np.power(y_pred - y_val_clean, 2), axis=1)
    loss = np.sqrt(np.mean(squared_error))
    losses.append(loss)
    print(f"k: {k}, loss: {loss:.4f}")

k: 1, loss: 15.2615
k: 3, loss: 13.5857
k: 5, loss: 13.4398
k: 10, loss: 13.4210
k: 20, loss: 13.9002
k: 50, loss: 14.7551


As expected, the model is worse when it has to predict more points, as the predicting picture is likely to be more different from the training picture, when you consider all of the keypoints rather than just the nose.